# Project 3
### Tanushree Balaji (tb33857)
### Nicolay Huarancay (nh23865)
### Neel Sheeth (nds967)
### Noah Shimizu (ncs838)

In [1]:
import numpy as np
from IPython.display import clear_output
import tensorflow as tf
import time
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, Input, MaxPool2D

In [2]:
tf.test.gpu_device_name()

''

# board structure
instead of using colors, we'll use +1 for one player and -1 for the other player.

In [3]:
def update_board(board_temp,color,column):
    # this is a function that takes the current board status, a color, and a column and outputs the new board status
    # columns 0 - 6 are for putting a checker on the board: if column is full just return the current board...this should be forbidden by the player
    # columns 7 - 13 are for pulling a checker off the board: this does not check if removing the checker is allowed...
    # 
    # the color input should be either 'plus' or 'minus'
    
    board = board_temp.copy()
    ncol = board.shape[1]
    nrow = board.shape[0]
    if column < ncol: # drop a checker on the board
        row = -1
        # start by assuming you can't add to the column
        # loop through the rows checking if you can go to each row or not
        for check in range(nrow):
            if (board[check,column]!=0):
                break # if this row is occupied, you're done
            else: # otherwise, you can go on this row!
                row += 1

        if row >= 0: # if you can add to the column
            if color == 'plus': # check the color
                board[row,column] = 1
            else:
                board[row,column] = -1
        return board
    else:
        column -= ncol
        if column >= ncol:
            return board # can't play anything bigger than 13...
        board[1:,column] = board[:-1,column].copy()
        board[0,column] = 0
        return board

In [4]:
board = np.zeros((6,7))
board = update_board(board,'plus',3)
print(board)
print('---------------------')
board = update_board(board,'minus',3)
print(board)
print('---------------------')
board = update_board(board,'plus',10)
print(board)

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]]
---------------------
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]]
---------------------
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]]


In [5]:
def check_for_win(board):
    # this function checks to see if anyone has won on the given board
    nrow = board.shape[0]
    ncol = board.shape[1]
    winner = 'nobody'
    for row in range(nrow):
        for col in range(ncol):
            # check for vertical winners
            if row <= (nrow-4): # can't have a column go from rows 4-7...
                if np.sum(board[row:(row+4),col])==4:
                    winner = 'v-plus'
                    return winner
                elif np.sum(board[row:(row+4),col])==-4:
                    winner = 'v-minus'
                    return winner
            # check for horizontal winners
            if col <= (ncol-4):
                if np.sum(board[row,col:(col+4)])==4:
                    winner = 'h-plus'
                    return winner
                elif np.sum(board[row,col:(col+4)])==-4:
                    winner = 'h-minus'
                    return winner
            # check for top left to bottom right diagonal winners
            if (row <= (nrow-4)) and (col <= (ncol-4)):
                if np.sum(board[range(row,row+4),range(col,col+4)])==4:
                    winner = 'd-plus'
                    return winner
                elif np.sum(board[range(row,row+4),range(col,col+4)])==-4:
                    winner = 'd-minus'
                    return winner
            # check for top right to bottom left diagonal winners
            if (row <= (nrow-4)) and (col >= 3):
                if np.sum(board[range(row,row+4),range(col,col-4,-1)])==4:
                    winner = 'd-plus'
                    return winner
                elif np.sum(board[range(row,row+4),range(col,col-4,-1)])==-4:
                    winner = 'd-minus'
                    return winner
    return winner

In [6]:
def display_board(board):
    # this function displays the board as ascii using X for +1 and O for -1
    clear_output()
    horizontal_line = '-'*(7*5+8)
    blank_line = '|'+' '*5
    blank_line *= 7
    blank_line += '|'
    print(horizontal_line)
    for row in range(6):
        print(blank_line)
        this_line = '|'
        for col in range(7):
            if board[row,col] == 0:
                this_line += ' '*5 + '|'
            elif board[row,col] == 1:
                this_line += '  X  |'
            else:
                this_line += '  O  |'
        print(this_line)
        print(blank_line)
        print(horizontal_line)

            

In [7]:
def is_legal(board, player, move):
    player = {"plus": 1, "minus": -1}[player]
    if(move < 7):
        return board[0,move] == 0
    return board[-1, move%7] == player

def where_legal(board, player):
    # Return indices of where legal to move
    player = {"plus": 1, "minus": -1}[player]
    answer = np.zeros(14)
    answer[:7] = board[0] == 0
    answer[7:] = board[-1] == player
    return answer

In [8]:
# this is how you can play a game
winner = 'nobody'
board = np.zeros((6,7))
display_board(board)
player = 'plus'
while winner == 'nobody':
    display_board(board)
    legal_move = False
    while(not legal_move):
        move = input('Pick a move (0-13) for player '+player+': ')
        move = int(move)
        legal_move = is_legal(board, player, move)
    board = update_board(board,player,move)
    winner = check_for_win(board)
    if player == 'plus':
        player = 'minus'
    else:
        player = 'plus'
print('The winner is '+winner)
    
    

-------------------------------------------
|     |     |     |     |     |     |     |
|     |     |     |     |     |     |     |
|     |     |     |     |     |     |     |
-------------------------------------------
|     |     |     |     |     |     |     |
|     |     |     |     |     |     |     |
|     |     |     |     |     |     |     |
-------------------------------------------
|     |     |     |     |     |     |     |
|     |     |     |     |     |     |     |
|     |     |     |     |     |     |     |
-------------------------------------------
|     |     |     |     |     |     |     |
|  O  |  X  |     |     |     |     |     |
|     |     |     |     |     |     |     |
-------------------------------------------
|     |     |     |     |     |     |     |
|  O  |  X  |     |     |     |     |     |
|     |     |     |     |     |     |     |
-------------------------------------------
|     |     |     |     |     |     |     |
|  O  |  X  |     |     |     | 

### Various Model Architectures that were tested

In [35]:
def create_model(height,width,channels):
    # we cannot simply have 3 output nodes because we want to put a weight on each node's impact to the objective
    # that is different for each data point.  the only way to achieve this is to have 3 output layers, each having 1 node
    # the effect is the same, just the way TF/keras handles weights is different
    imp = Input(shape=(height,width,channels))
    mid = Conv2D(32, (2,2), activation='tanh')(imp)
    mid = Conv2D(64, (2,2), activation='tanh')(mid)
    mid = Conv2D(64, (2,2), activation='tanh')(mid)
    mid = Flatten()(mid)
    mid = Dense(128,activation='relu')(mid) # could also use a tanh activation function...all VFs are between -1 to 1
    mid = Dense(64,activation='relu')(mid) # could also use a tanh activation function...all VFs are between -1 to 1
    mid = Dense(64,activation='relu')(mid) # could also use a tanh activation function...all VFs are between -1 to 1
    out = Dense(14, activation='linear')(mid) # could also use a tanh activation function...all VFs are between -1 to 1
    model = Model(imp,out) 
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),loss='mean_squared_error')
    
    return model

In [200]:
def create_model2(height,width,channels):
    # we cannot simply have 3 output nodes because we want to put a weight on each node's impact to the objective
    # that is different for each data point.  the only way to achieve this is to have 3 output layers, each having 1 node
    # the effect is the same, just the way TF/keras handles weights is different
    imp = Input(shape=(height,width,channels))
    mid = Conv2D(64, (4,4), activation='relu')(imp)
    mid = Conv2D(64, (2,2), activation='relu')(mid)
    mid = Conv2D(64, (2,2), activation='relu')(mid)
    mid = Flatten()(mid)
    mid = Dense(100,activation='relu')(mid)
    mid = Dense(64,activation='relu')(mid) 
    out = Dense(14, activation='tanh')(mid) # could also use a tanh activation function...all VFs are between -1 to 1
    model = Model(imp,out) 
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),loss='mean_squared_error')
    
    return model

In [442]:
def create_model3(height,width,channels):
    # we cannot simply have 3 output nodes because we want to put a weight on each node's impact to the objective
    # that is different for each data point.  the only way to achieve this is to have 3 output layers, each having 1 node
    # the effect is the same, just the way TF/keras handles weights is different
    imp = Input(shape=(height,width,channels))
    mid = Conv2D(50, (4,4), activation='relu')(imp)
    mid = Conv2D(75, (2,2), activation='relu')(mid)
    mid = Conv2D(100, (2,2), activation='relu')(mid)
    mid = Flatten()(mid)
    mid = Dense(64,activation='relu')(mid)
    mid = Dense(64,activation='relu')(mid) 
    out = Dense(14, activation='tanh')(mid) # could also use a tanh activation function...all VFs are between -1 to 1
    model = Model(imp,out) 
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),loss='mean_squared_error')
    
    return model

### Some helper functions for finding the best move and best value

In [14]:
def best_move(board, player):
    # Returns best move for a model
    mod = players[player]
    values = mod(np.expand_dims(board,[0,3]),training=False)
    legal_values = values + 2*where_legal(board, player)
    return np.argmax(legal_values[0])

def best_value(board, player):
    # Returns value for the best move of a model
    mod = players[player]
    values = mod(np.expand_dims(board,[0,3]),training=False)
    legal_values = values + 2*where_legal(board, player)
    return np.max(legal_values[0]) - 2

### Setup the training

In [444]:
warmupframes = 10000
max_frames = 10000000+warmupframes
tot_frames = 0
anneal1 = 10000
anneal2 = 100000
batch_size = 32
buffn = 5000

len_buff = 0
ep0 = 0.95
ep1 = 0.1
ep2 = 0.05
ep = ep0
dep1 = (ep0-ep1)/anneal1
dep2 = (ep1-ep2)/anneal2
buffer = {"plus":[],
          "minus":[]}
buffer_nf = {"plus":[],
              "minus":[]}
buffer_ac = {"plus":[],
              "minus":[]}
buffer_rw = {"plus":[],
              "minus":[]}
mse_loss = tf.keras.losses.MeanSquaredError()

game_num = 0

In [621]:
batch_size = 32
ep2 = 0.05
ep = 0.05

In [446]:
frames_to_net = 1              # how many previous frames will we feed the NN
possible_actions = np.arange(14)
delt = 0.95
player1 = create_model3(6,7,frames_to_net)
player1.call = tf.function(player1.call,experimental_relax_shapes=True,reduce_retracing=True)

player2 = create_model3(6,7,frames_to_net)
player2.call = tf.function(player2.call,experimental_relax_shapes=True,reduce_retracing=True)

players = {"plus":player1, "minus":player2 }

player1.summary()

Model: "model_56"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_57 (InputLayer)       [(None, 6, 7, 1)]         0         
                                                                 
 conv2d_168 (Conv2D)         (None, 3, 4, 50)          850       
                                                                 
 conv2d_169 (Conv2D)         (None, 2, 3, 75)          15075     
                                                                 
 conv2d_170 (Conv2D)         (None, 1, 2, 100)         30100     
                                                                 
 flatten_56 (Flatten)        (None, 200)               0         
                                                                 
 dense_168 (Dense)           (None, 64)                12864     
                                                                 
 dense_169 (Dense)           (None, 64)                416

### Train

In [635]:
while tot_frames < max_frames:
    winner = 'nobody'
    board = np.zeros((6,7))
    player = 'plus'
    frames_this_game = 0
    st = time.time()
    game_num += 1
    feed = np.zeros((6,7,1))
    while winner == 'nobody':
        # Select which model ur looking at
        mod = players[player]
        
        # If random, make random move
        if np.random.random() < ep:
            possible_moves = where_legal(board,player).nonzero()[0]
            move = np.random.choice(possible_moves)
        # Otherwise, pick the best move
        else:
            move = best_move(board, player)
            
        # update frames
        buffer[player].append(feed.copy())
        
        # Make moves
        board = update_board(board,player,move)
        feed[:,:,0] = board
        
        # Check for winner
        winner = check_for_win(board)
        
        # update new frames
        buffer_nf[player].append(feed.copy())
        buffer_ac[player].append(move)
        buffer_rw[player].append(1 if winner[2:] == player else -1 if winner != "nobody" else 0)
        
        #update framecount
        tot_frames += 1
        
        # update weights and biases
        if tot_frames > warmupframes:
            # anneal epsilon
            if ep > ep1:
                ep -= dep1
            elif ep > ep2:
                ep -= dep2
            for possible_player in ["plus", "minus"]:
                # Select which model ur looking at
                mod = players[possible_player]
                other_player = "plus" if possible_player == "minus" else "minus"
                opp_mod = players[other_player]
                # how many frames are there to choose from
                lbuff = len(buffer[possible_player])
                
                # Wait the chances of choosing each sample
                p = (np.array(buffer_rw[possible_player]) != 0)*2 + 1
                p = p/np.sum(p)
                
                # Sample random frames
                which_choose = np.random.choice(lbuff,batch_size,replace=False, p = p)
                
                # Grab current frames
                current_frames = np.array(buffer[possible_player])[which_choose]
                next_frames = np.array(buffer_nf[possible_player])[which_choose]
                actions = np.zeros((batch_size,2))
                actions[:,0] = np.arange(batch_size)
                actions[:,1] = np.array(buffer_ac[possible_player])[which_choose]
                rewards = np.array(buffer_rw[possible_player])[which_choose]
                

                # Construct next frames
                # Find opponent's rewards for moves
                op_rewards = opp_mod(next_frames, training = False)
                
                # Check legal moves
                legal_moves = np.array([where_legal(next_frame,other_player) for next_frame in np.squeeze(next_frames,3)])
                
                # Force opponent to only pick from legal moves
                op_rewards = opp_mod(next_frames) + 2 * legal_moves
                
                # Find opponent's best move
                op_moves = tf.argmax(op_rewards, axis = 1)
                # Play opponent's moves
                next_frames = np.array([update_board(next_frames[i], other_player, op_moves[i]) for i in range(batch_size)])
                
                # Check for winner after opponent's moves
                pos_win = [check_for_win(frame)[2:] for frame in np.squeeze(next_frames,3)]
                win = np.array([-1 if won == other_player else 1 if won == possible_player else 0 for won in pos_win])
                
                # Update rewards for if opponent forces win
                rewards = rewards*(rewards != 0) + win*(rewards == 0)
                
                # Return rewards
                target_vals = mod(next_frames)
                # Make it so target only exists where legal moves exist
                where_legal_targets = np.array([where_legal(np.squeeze(next_frame), possible_player) for next_frame in next_frames])
                target_vals = np.array(target_vals) + where_legal_targets * 2
                target = tf.reduce_max(target_vals,axis = 1) - 2
                y = delt*target*(rewards==0) + rewards
                
                
                # gradienttape is pretty cool
                # if you ONLY do tf operations inside the tape, then TF can auto-differentiate!
                with tf.GradientTape() as tape:
                    # make prediction on x data
                    pred = tf.gather_nd(mod(current_frames,training=False),actions.astype('int32'))
                    # tf.gather_nd is the tf operation to evaluate only particular entries in the array, as specified by the actions list
                    loss = mse_loss(y,pred) # calculate mse of truth - prediction
                # calculate the gradient of the stuff inside the tape
                gradient = tape.gradient(loss,mod.trainable_variables)
                # do 1 step of sgd (with the optimizer specified above)
                mod.optimizer.apply_gradients(zip(gradient,mod.trainable_variables))
        frames_this_game += 1
        # swap player
        player = 'plus' if player == 'minus' else 'minus'
        
    # Check if buffer needs to be updated.
    for possible_player in ["plus", "minus"]:
        player_buffer = buffer[possible_player]
        lbuff = len(player_buffer)
        if lbuff > buffn:
            excess = lbuff - buffn
            buffer[possible_player] = player_buffer[excess:].copy()
            buffer_nf[possible_player] = buffer_nf[possible_player][excess:].copy()
            buffer_rw[possible_player] = buffer_rw[possible_player][excess:].copy()
            buffer_ac[possible_player] = buffer_ac[possible_player][excess:].copy()
            len_buff = len(player_buffer)
    print(game_num,winner,ep,tot_frames,time.time()-st)

7262 v-plus 0.05 160577 9.93608570098877
7263 d-plus 0.05 160598 6.3418707847595215
7264 v-plus 0.05 160623 7.654531717300415
7265 v-minus 0.05 160643 5.791690826416016
7266 h-plus 0.05 160666 6.389845371246338
7267 v-minus 0.05 160688 6.1801464557647705
7268 v-minus 0.05 160714 7.33769416809082
7269 d-plus 0.05 160725 3.158633232116699
7270 d-plus 0.05 160736 3.1688833236694336
7271 d-plus 0.05 160747 3.262704372406006
7272 d-minus 0.05 160767 5.894834756851196
7273 h-minus 0.05 160787 5.707542896270752
7274 v-minus 0.05 160809 6.274221897125244
7275 h-plus 0.05 160836 7.681342363357544
7276 v-minus 0.05 160854 5.130181789398193
7277 v-minus 0.05 160872 5.180473327636719
7278 d-minus 0.05 160886 4.021931886672974
7279 d-plus 0.05 160917 8.916974306106567
7280 d-plus 0.05 160954 10.657253503799438
7281 v-plus 0.05 160975 5.9375903606414795
7282 d-plus 0.05 160988 3.6627206802368164
7283 d-plus 0.05 161001 3.707719326019287
7284 d-plus 0.05 161028 7.695607900619507
7285 h-minus 0.05 161

KeyboardInterrupt: 

best_value(player, new_board)

In [ ]:
player1.save("player1_final_2.h5")

In [ ]:
player2.save("player2_final_2.h5")

## Read in our trained models

In [26]:
player1 = tf.keras.models.load_model("player1_final_2.h5")
player1.call = tf.function(player1.call,experimental_relax_shapes=True,reduce_retracing=True)

player2 = tf.keras.models.load_model("player2_final_2.h5")
player2.call = tf.function(player2.call,experimental_relax_shapes=True,reduce_retracing=True)

players = {"plus":player1, "minus":player2 }

player1.summary()

Model: "model_56"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_57 (InputLayer)       [(None, 6, 7, 1)]         0         
                                                                 
 conv2d_168 (Conv2D)         (None, 3, 4, 50)          850       
                                                                 
 conv2d_169 (Conv2D)         (None, 2, 3, 75)          15075     
                                                                 
 conv2d_170 (Conv2D)         (None, 1, 2, 100)         30100     
                                                                 
 flatten_56 (Flatten)        (None, 200)               0         
                                                                 
 dense_168 (Dense)           (None, 64)                12864     
                                                                 
 dense_169 (Dense)           (None, 64)                416

## Has our model play a game against each other

In [15]:
def play_game(model1, model2):
    # this is how you can play a game
    winner = 'nobody'
    board = np.zeros((6,7))
    display_board(board)
    player = 'plus'
    mod = model1
    while winner == 'nobody':
        move = best_move(board, player)
        vf = mod(np.expand_dims(board,[0,3]),training=False)
        vf = np.array(vf)
        move = np.argmax(vf + 2.1*where_legal(board,player))
        time.sleep(2)
        board = update_board(board,player,move)
        display_board(board)
        print(player, move)
        print(vf)
        winner = check_for_win(board)
        if player == 'plus':
            player = 'minus'
            mod = model2
        else:
            player = 'plus'
            mod = model1
    print(winner)

In [16]:
play_game(player1, player2)

-------------------------------------------
|     |     |     |     |     |     |     |
|     |     |     |     |  O  |  X  |     |
|     |     |     |     |     |     |     |
-------------------------------------------
|     |     |     |     |     |     |     |
|     |     |     |     |  X  |  O  |     |
|     |     |     |     |     |     |     |
-------------------------------------------
|     |     |     |     |     |     |     |
|     |     |     |  O  |  O  |  X  |  X  |
|     |     |     |     |     |     |     |
-------------------------------------------
|     |     |     |     |     |     |     |
|     |     |     |  X  |  O  |  X  |  O  |
|     |     |     |     |     |     |     |
-------------------------------------------
|     |     |     |     |     |     |     |
|     |     |     |  O  |  X  |  O  |  X  |
|     |     |     |     |     |     |     |
-------------------------------------------
|     |     |     |     |     |     |     |
|  O  |     |     |  X  |  X  | 

## Have the human play against our model

In [17]:
def play_model(user, opposing_model):
    # this is how you can play a game
    winner = 'nobody'
    board = np.zeros((6,7))
    display_board(board)
    player = 'plus'
    vf = 0
    while winner == 'nobody':
        move = best_move(board, player)
        if(user == player):
            move = int(input("what move:"))
        else:
            vf = opposing_model(np.expand_dims(board,[0,3]),training=False)
            vf = np.array(vf)
            move = np.argmax(vf + 2.1*where_legal(board,player))
        board = update_board(board,player,move)
        display_board(board)
        print(vf)
        print(move)
        winner = check_for_win(board)
        if player == 'plus':
            player = 'minus'
        else:
            player = 'plus'
    print(winner)

In [18]:
play_model("minus",player1)

-------------------------------------------
|     |     |     |     |     |     |     |
|     |     |     |     |     |     |     |
|     |     |     |     |     |     |     |
-------------------------------------------
|     |     |     |     |     |     |     |
|     |     |     |     |     |     |     |
|     |     |     |     |     |     |     |
-------------------------------------------
|     |     |     |     |     |     |     |
|     |     |     |     |  X  |     |     |
|     |     |     |     |     |     |     |
-------------------------------------------
|     |     |     |     |     |     |     |
|  O  |     |     |     |  X  |     |     |
|     |     |     |     |     |     |     |
-------------------------------------------
|     |     |     |     |     |     |     |
|  O  |     |     |     |  X  |     |     |
|     |     |     |     |     |     |     |
-------------------------------------------
|     |     |     |     |     |     |     |
|  O  |  O  |  O  |  X  |  X  | 

## Has our model play against random

In [21]:
def play_against_random(model, model_plays, wait = False, display = False):
    # this is how you can play a game
    winner = 'nobody'
    board = np.zeros((6,7))
    if(display):
        display_board(board)
    player = 'plus'
    while winner == 'nobody':
        legal_move = False
        feed = np.zeros((6,7,1))
        feed[:,:,0] = board
        if(wait):
            time.sleep(1)
        while(not legal_move):
            if(player == model_plays):
                vf = model(np.expand_dims(board,[0,3]),training=False)
                vf = np.array(vf)
            else:
                vf = np.random.rand(1,14)
            while(not legal_move):
                move = np.argmax(vf)
                legal_move = is_legal(board, player, move)
                vf[:,move] = -100
        board = update_board(board,player,move)
        if(display):
            display_board(board)
        winner = check_for_win(board)
        if player == 'plus':
            player = 'minus'
        else:
            player = 'plus'
    return winner[2:] == model_plays

## Tests our model's accuracy against random

In [24]:
np.mean([play_against_random(player1, 'plus') for i in range(1000)])

0.987

In [637]:
np.mean([play_against_random(player2, 'minus') for i in range(1000)])

0.963